## Execute the steps manually in the notebook ##

### Fill table below to submit batch job

In [ ]:

# User-defined input table
user_data = {
    "username": "your_username",
    "password": "your_password",
    "client_id": "example_client_id",
    "project_id": "example_project_id",
    "dataset_id": "example_dataset_id",
    "table_id": "example_table_id",
    "table_key": "{}",
    "row_count": 100,
    "request_column": 0,
    "response_column": 1,
    "llm_model": "gpt-3.5",
    "prompt_prefix": "Hello,",
    "prompt_postfix": "Thanks!",
}


### These are helper functions  and libraries required to submit a job 

In [ ]:
import requests
import time
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:

def retrieve_api_key():
    username = user_data["username"]
    password = user_data["password"]

    auth_data = {"username": username, "password": password}
    try:
        response = requests.post(f"{API_BASE_URL}/get_api_key", json=auth_data, headers=headers, timeout=10)
        if response.status_code == 200:
            api_key = response.json().get("api_key")
            logging.info(f"API key retrieved successfully: {api_key}")
            headers["x-api-key"] = api_key
            return api_key
        else:
            logging.error(f"Error retrieving API key: {response.status_code} - {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error while retrieving API key: {e}")
        return None


In [ ]:

def get_job_data():
    return {
        "client_id": user_data["client_id"],
        "project_id": user_data["project_id"],
        "dataset_id": user_data["dataset_id"],
        "table_id": user_data["table_id"],
        "table_key": user_data["table_key"],
        "row_count": user_data["row_count"],
        "request_column": user_data["request_column"],
        "response_column": user_data["response_column"],
        "llm_model": user_data["llm_model"],
        "prompt_prefix": user_data["prompt_prefix"],
        "prompt_postfix": user_data["prompt_postfix"],
    }


In [ ]:
def submit_job():
    job_data = get_job_data_from_user()

    try:
        response = requests.post(f"{API_BASE_URL}/submit_job", json=job_data, headers=headers, timeout=10)
        if response.status_code == 200:
            logging.info("Job submitted successfully.")
            return response.json()['job_id']
        else:
            logging.error(f"Error submitting job: {response.status_code} - {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error occurred: {e}")
        return None


In [ ]:
def check_job_status(job_id):
    try:
        response = requests.get(f"{API_BASE_URL}/job_status/{job_id}", headers=headers, timeout=10)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 404:
            logging.warning(f"Job with ID {job_id} not found.")
        else:
            logging.error(f"Failed to retrieve job status: {response.status_code} - {response.text}")
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error while checking job status: {e}")
    return None


### Ensure that the API URL is correct with the one hosted in the cloud

In [ ]:
API_BASE_URL = "https://flask-api-1069651367433.us-east4.run.app"
headers = {
    "Content-Type": "application/json",
}

## Script that will submit job

In [ ]:
# Step 1: Retrieve API Key
api_key = retrieve_api_key()

# Step 2: Submit a Job
if api_key:
    job_id = submit_job()
    if job_id:
        logging.info(f"Job ID: {job_id}")

        # Step 3: Poll Job Status
        backoff_time = 30
        max_backoff_time = 3600
        max_attempts = 10

        attempts = 0
        while attempts < max_attempts:
            status = check_job_status(job_id)
            if status:
                logging.info(f"Job status: {status['status']}")
                if status['status'] == "completed":
                    logging.info("Job completed successfully!")
                    break
                elif status['status'] == "processing":
                    logging.info("Job is still processing. Checking again in a few seconds...")
            else:
                logging.warning("Failed to retrieve job status. Retrying...")
            time.sleep(backoff_time)
            backoff_time = min(backoff_time * 2, max_backoff_time)
            attempts += 1

        if attempts == max_attempts:
            logging.error("Maximum number of attempts reached. Job status check terminated.")


In [ ]:

def check_job_status_manual():
    # Ensure `api_key` and `job_id` are already defined in the environment
    global api_key, job_id
    if not api_key or not job_id:
        raise ValueError("API key or Job ID not found. Please ensure the job has been initiated.")

    # URL for checking job status
    job_status_url = f"{API_BASE_URL}/jobs/{job_id}/status"
    try:
        # Make the GET request to fetch job status
        response = requests.get(job_status_url, headers={"x-api-key": api_key}, timeout=10)
        if response.status_code == 200:
            job_status = response.json()
            logging.info(f"Job Status: {job_status}")
            return job_status
        else:
            logging.error(f"Error fetching job status: {response.status_code} - {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error while checking job status: {e}")
        return None

# User Instructions:
# After running the setup and initiating a job, you can rerun this cell at any time to check the job status.


In [ ]:
check_job_status_manual()